<a href="https://colab.research.google.com/github/learn-programmers/programmers_kdt_II/blob/main/9%EC%A3%BC%EC%B0%A8_PySpark_%EA%B8%B0%EB%B3%B8_2%EC%9D%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     |████████████████████████████████| 204.2MB 67kB/s 
     |████████████████████████████████| 204kB 46.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=1ab6d28ae53ae0edaf7482161813b1135e98a3b423328132c43a9c9973b8c55e
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
!ls -tl

total 4
drwxr-xr-x 1 root root 4096 Jan 20 17:27 sample_data


In [3]:
!ls -tl sample_data

total 55504
-rw-r--r-- 1 root root 18289443 Jan 20 17:27 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Jan 20 17:27 mnist_train_small.csv
-rw-r--r-- 1 root root   301141 Jan 20 17:27 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Jan 20 17:27 california_housing_train.csv
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md


**Spark Session:** SparkSession은 Spark 2.0부터 엔트리 포인트로 사용된다. 그 이전에는 SparkContext가 사용되었다. SparkSession을 이용해 RDD, 데이터 프레임등을 만든다. SparkSession은 SparkSession.builder를 호출하여 생성하며 다양한 함수들을 통해 세부 설정이 가능하다

In [4]:
from pyspark.sql import SparkSession

# spark 세션 오브젝트 만들기
spark = SparkSession.builder\
        .master("local[*]")\ # master다음의 인자는 내가 사용하고싶은 spark 클러스터 이름을 준다. local은 내 로컬 spark stanalone을 쓰겠다는 것. *은 서버의 모든 cpu를 쓴다는 것이다.
        .appName('PySpark_Tutorial')\
        .getOrCreate() # PySpark_Tutorial와 같은거 있으면 불러오고 아니면 새로 만들어라

In [5]:
spark # 버전, 마스터는 로컬, 모든 cpu를 씀, 앱이름 출력

**Python 객체를 RDD로 변환해보기**

**1> Python 리스트 생성**

In [6]:
name_list_json = [ '{"name": "keeyong"}', '{"name": "benjamin"}', '{"name": "claire"}' ]

In [8]:
for n in name_list_json: # 3개의 json 오브젝트 출력
  print(n)

{"name": "keeyong"}
{"name": "benjamin"}
{"name": "claire"}


In [9]:
import json

for n in name_list_json:
  jn = json.loads(n) # 파이썬 딕셔너리로 바꾼 뒤
  print(jn["name"]) # 딕셔너리의 name만 출력

keeyong
benjamin
claire


**2> 파이썬 리스트를 RDD로 변환. RDD로 변환되는 순간 Spark 클러스터의 서버들에 데이터가 나눠 저장됨 (파티션). 또한 Lazy Execution이 된다는 점 기억**

In [11]:
rdd = spark.sparkContext.parallelize(name_list_json)

In [12]:
rdd

ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:262

In [13]:
rdd.count() # rdd에 들어있는 레코드 수

3

In [14]:
# rdd의 functional프로그래밍 map을 통해 해보기
# map은 rdd 전체의 엘리먼트들에게 람다펑션을 취해서 새로운 rdd로 만든다.
# string이었던 rdd의 원소들이 파이썬의 딕셔너리 형태로 바꿔서 parse_rdd에 저장된다.
parsed_rdd = rdd.map(lambda el:json.loads(el))

In [15]:
parsed_rdd 

PythonRDD[3] at RDD at PythonRDD.scala:53

In [16]:
parsed_rdd.collect()
# collect해서 출력하면 rdd의 정보들이 수집되어 파이썬 프로그램으로 넘어와 출력된다.

[{'name': 'keeyong'}, {'name': 'benjamin'}, {'name': 'claire'}]

In [17]:
# rdd를 람다펑션으로 이름만 뽑도록 해보자.  
parsed_name_rdd = rdd.map(lambda el:json.loads(el)["name"])

In [18]:
parsed_name_rdd.collect() # 이름만 나오는 것을 확인할 수 있다.
# 파이썬으로는 1000만개의 데이터에서 이름만 뽑을 수 없다.
# 하지만 spark의 rdd를 이용해 1000만개더라도 이름만 뽑을 수 있다.
# 다만 collect로 받아올 때는 받아올 데이터가 작아야 한다! 파이썬으로 넘겨오면서 오류날 수 있다.

['keeyong', 'benjamin', 'claire']

**파이썬 리스트를 데이터프레임으로 변환하기**

In [19]:
from pyspark.sql.types import StringType

# name_list_json라는 파이썬 리스트를 이번에는 데이터프레임으로 바꾸자.
# 스트링임을 알려줘야 한다. StringType()
df = spark.createDataFrame(name_list_json, StringType())

In [20]:
df.count()

3

In [21]:
df.printSchema()
# rdd와 데이터프레임의 다른 점!
# 컬럼이 생기고, 타입정보는 모르기때문에 위에서 StringType()로 명시한다.
# 위에서 필드 이름을 안줬기 때문에 기본적으로 필드 이름이 value가 된다.

root
 |-- value: string (nullable = true)



In [22]:
df.select('*').collect()
# 여러 필드 중 일부 필드만 선택해볼 수 있다.
# *은 모든 필드 선택

[Row(value='{"name": "keeyong"}'),
 Row(value='{"name": "benjamin"}'),
 Row(value='{"name": "claire"}')]

In [ ]:
df.select('value').collect()
# 여기서는 필드가 하나이기 때문에 위의 *를 한 결과와 동일하다.

[Row(value='{"name": "keeyong"}'),
 Row(value='{"name": "benjamin"}'),
 Row(value='{"name": "claire"}')]

In [24]:
from pyspark.sql import Row

# 필드(컬럼)에 이름을 지정해줄 수 있다.
row = Row("name") # Or some other column name

# rdd의 경우 이름이 없기 때문에 row라는 모듈을 통해 이름을 주면서 데이터프레임으로 바꾼다.
df_name = parsed_name_rdd.map(row).toDF()

In [25]:
df_name.printSchema()
# 필드 이름이 바뀐 것을 확인할 수 있다.

root
 |-- name: string (nullable = true)



In [28]:
df_name.select('name').collect()
# name 필드의 내용을 볼 수 있다.

[Row(name='keeyong'), Row(name='benjamin'), Row(name='claire')]